# 🔧 Feature Engineering V2: Crohn vs CU Separado

**Objetivo:** Crear features para predecir el riesgo de brotes en pacientes con Crohn y CU **por separado**

**Input:** 
- `../data/processed/crohn_filtered.csv` (del notebook 01 V2)
- `../data/processed/cu_filtered.csv` (del notebook 01 V2)

**Output:** Datasets estructurados con features para entrenamiento:
- `../data/processed/crohn/ml_dataset.csv`
- `../data/processed/cu/ml_dataset.csv`

**Mejoras vs V1:**
- Procesa Crohn y CU independientemente
- Permite ajustar severity scoring para cada tipo
- Genera datasets separados para modelos específicos

**Autor:** Asier Ortiz García  
**Fecha:** Noviembre 2025

## 📦 Imports y Configuración

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from pathlib import Path
import warnings
import json
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 10
%matplotlib inline

# Crear directorios
Path('../data/processed/crohn').mkdir(parents=True, exist_ok=True)
Path('../data/processed/cu').mkdir(parents=True, exist_ok=True)
Path('../docs/figures').mkdir(parents=True, exist_ok=True)

print("=" * 80)
print("FEATURE ENGINEERING V2: Crohn vs CU")
print("=" * 80)

## 🔄 Funciones Reutilizables

In [ ]:
# Mapeo de síntomas
SYMPTOM_MAPPING = {
    'abdominal_pain': [
        'abdominal pain', 'stomach pain', 'belly pain', 'cramping',
        'abdominal cramps', 'stomach cramps', 'pain'
    ],
    'diarrhea': [
        'diarrhea', 'loose stools', 'watery stools', 'frequent bowel movements',
        'urgent bowel movements', 'urgency'
    ],
    'fatigue': [
        'fatigue', 'tired', 'exhaustion', 'tiredness', 'weakness',
        'lack of energy', 'low energy'
    ],
    'fever': [
        'fever', 'high temperature', 'chills'
    ],
    'blood_in_stool': [
        'blood in stool', 'bloody stool', 'rectal bleeding', 'bleeding'
    ],
    'nausea': [
        'nausea', 'nauseous', 'feeling sick', 'queasiness', 'vomiting'
    ]
}

# Pesos de síntomas (ajustables por tipo de IBD si es necesario)
SYMPTOM_WEIGHTS_CROHN = {
    'abdominal_pain': 0.25,
    'diarrhea': 0.25,
    'fatigue': 0.15,
    'blood_in_stool': 0.20,
    'fever': 0.10,
    'nausea': 0.05
}

SYMPTOM_WEIGHTS_UC = {
    'abdominal_pain': 0.20,
    'diarrhea': 0.25,
    'fatigue': 0.15,
    'blood_in_stool': 0.25,  # Más peso en CU
    'fever': 0.10,
    'nausea': 0.05
}

def categorize_symptom(symptom_name):
    """Categoriza un síntoma en una de nuestras categorías."""
    if pd.isna(symptom_name):
        return None
    
    symptom_lower = str(symptom_name).lower()
    
    for category, keywords in SYMPTOM_MAPPING.items():
        for keyword in keywords:
            if keyword in symptom_lower:
                return category
    
    return 'other'

def classify_risk(score):
    """Clasificar severity score en risk level."""
    if score < 0.3:
        return 'low'
    elif score < 0.6:
        return 'medium'
    else:
        return 'high'

print("✓ Funciones definidas")

In [ ]:
def process_ibd_data(df, ibd_type='crohn', symptom_weights=None):
    """
    Procesar datos de un tipo de IBD específico.
    
    Args:
        df: DataFrame con datos filtrados
        ibd_type: 'crohn' o 'cu'
        symptom_weights: Dict con pesos para cada síntoma
    
    Returns:
        ml_dataset: DataFrame listo para entrenamiento
    """
    print(f"\n{'='*80}")
    print(f"PROCESANDO: {ibd_type.upper()}")
    print(f"{'='*80}\n")
    
    if symptom_weights is None:
        symptom_weights = SYMPTOM_WEIGHTS_CROHN
    
    # 1. Procesar síntomas
    print("1️⃣ Procesando síntomas...")
    symptoms = df[df['trackable_type'] == 'Symptom'].copy()
    symptoms['symptom_category'] = symptoms['trackable_name'].apply(categorize_symptom)
    symptoms['value_numeric'] = pd.to_numeric(symptoms['trackable_value'], errors='coerce')
    
    # Filtrar valores válidos y síntomas principales
    symptoms_clean = symptoms[
        (symptoms['value_numeric'] >= 0) & 
        (symptoms['value_numeric'] <= 4) &
        (symptoms['symptom_category'].isin(list(SYMPTOM_MAPPING.keys())))
    ].copy()
    
    symptoms_clean['severity_normalized'] = symptoms_clean['value_numeric'] / 4.0
    
    print(f"  Síntomas procesados: {len(symptoms_clean):,}")
    print(f"  Distribución:")
    print(f"  {symptoms_clean['symptom_category'].value_counts()}")
    
    # 2. Agregar por usuario y fecha
    print("\n2️⃣ Agregando por usuario-fecha...")
    daily_symptoms = symptoms_clean.groupby(
        ['user_id', 'checkin_date', 'symptom_category']
    )['severity_normalized'].max().reset_index()
    
    daily_pivot = daily_symptoms.pivot_table(
        index=['user_id', 'checkin_date'],
        columns='symptom_category',
        values='severity_normalized',
        fill_value=0.0
    ).reset_index()
    
    print(f"  Registros diarios: {len(daily_pivot):,}")
    print(f"  Usuarios: {daily_pivot['user_id'].nunique():,}")
    
    # 3. Calcular severity score
    print("\n3️⃣ Calculando severity score...")
    symptom_cols = [col for col in daily_pivot.columns if col in SYMPTOM_MAPPING.keys()]
    daily_pivot['severity_score'] = 0.0
    
    for symptom in symptom_cols:
        weight = symptom_weights.get(symptom, 0.1)
        daily_pivot['severity_score'] += daily_pivot[symptom] * weight
    
    daily_pivot['risk_level'] = daily_pivot['severity_score'].apply(classify_risk)
    
    print(f"  Distribución risk levels:")
    print(f"  {daily_pivot['risk_level'].value_counts()}")
    
    # 4. Features demográficos
    print("\n4️⃣ Añadiendo features demográficos...")
    demographics = df[['user_id', 'age', 'sex']].drop_duplicates('user_id')
    daily_pivot = daily_pivot.merge(demographics, on='user_id', how='left')
    
    daily_pivot['age'].fillna(daily_pivot['age'].median(), inplace=True)
    daily_pivot['sex'].fillna('unknown', inplace=True)
    
    gender_map = {'male': 'M', 'female': 'F', 'unknown': 'O', 'other': 'O'}
    daily_pivot['gender'] = daily_pivot['sex'].map(gender_map).fillna('O')
    
    print(f"  Age: media={daily_pivot['age'].mean():.1f}, mediana={daily_pivot['age'].median():.1f}")
    print(f"  Gender: {daily_pivot['gender'].value_counts().to_dict()}")
    
    # 5. Disease duration
    print("\n5️⃣ Calculando disease duration...")
    first_checkin = df.groupby('user_id')['checkin_date'].min().reset_index()
    first_checkin.columns = ['user_id', 'first_checkin']
    daily_pivot = daily_pivot.merge(first_checkin, on='user_id', how='left')
    
    daily_pivot['days_since_first_checkin'] = (
        daily_pivot['checkin_date'] - daily_pivot['first_checkin']
    ).dt.days
    daily_pivot['disease_duration_years'] = (daily_pivot['days_since_first_checkin'] / 365.25).fillna(0)
    
    print(f"  Media: {daily_pivot['disease_duration_years'].mean():.2f} años")
    
    # 6. Previous flares y days since last flare
    print("\n6️⃣ Calculando historial de brotes...")
    daily_pivot = daily_pivot.sort_values(['user_id', 'checkin_date'])
    daily_pivot['is_flare_day'] = (daily_pivot['risk_level'] == 'high').astype(int)
    daily_pivot['cumulative_flare_days'] = daily_pivot.groupby('user_id')['is_flare_day'].cumsum()
    daily_pivot['previous_flares'] = (daily_pivot['cumulative_flare_days'] / 7).astype(int)
    
    def calc_days_since_flare(group):
        group = group.sort_values('checkin_date')
        last_flare_date = None
        days_since = []
        
        for idx, row in group.iterrows():
            if last_flare_date is None:
                days_since.append(365)
            else:
                days = (row['checkin_date'] - last_flare_date).days
                days_since.append(days)
            
            if row['risk_level'] == 'high':
                last_flare_date = row['checkin_date']
        
        group['last_flare_days_ago'] = days_since
        return group
    
    daily_pivot = daily_pivot.groupby('user_id', group_keys=False).apply(calc_days_since_flare)
    
    print(f"  Previous flares: media={daily_pivot['previous_flares'].mean():.2f}")
    print(f"  Days since last flare: media={daily_pivot['last_flare_days_ago'].mean():.1f}")
    
    # 7. Features temporales
    print("\n7️⃣ Añadiendo features temporales...")
    daily_pivot['month'] = daily_pivot['checkin_date'].dt.month
    daily_pivot['day_of_week'] = daily_pivot['checkin_date'].dt.dayofweek
    
    # 8. Dataset final
    print("\n8️⃣ Preparando dataset final...")
    symptom_features = [col for col in symptom_cols if col in daily_pivot.columns]
    demographic_features = ['age', 'gender']
    history_features = ['disease_duration_years', 'previous_flares', 'last_flare_days_ago']
    temporal_features = ['month', 'day_of_week']
    target = 'risk_level'
    
    all_features = symptom_features + demographic_features + history_features + temporal_features
    ml_dataset = daily_pivot[all_features + [target, 'user_id', 'checkin_date']].copy()
    
    # Imputar valores faltantes
    for col in all_features:
        if ml_dataset[col].dtype in ['float64', 'int64']:
            ml_dataset[col].fillna(ml_dataset[col].median(), inplace=True)
        else:
            if len(ml_dataset[col].mode()) > 0:
                ml_dataset[col].fillna(ml_dataset[col].mode()[0], inplace=True)
    
    print(f"  ✓ Dataset final: {len(ml_dataset):,} registros")
    print(f"  ✓ Features: {len(all_features)}")
    print(f"  ✓ Usuarios: {ml_dataset['user_id'].nunique():,}")
    
    # Metadata
    metadata = {
        'ibd_type': ibd_type,
        'features': all_features,
        'symptom_features': symptom_features,
        'demographic_features': demographic_features,
        'history_features': history_features,
        'temporal_features': temporal_features,
        'target': target,
        'n_samples': len(ml_dataset),
        'n_users': int(ml_dataset['user_id'].nunique()),
        'class_distribution': ml_dataset[target].value_counts().to_dict()
    }
    
    return ml_dataset, metadata

print("✓ Función de procesamiento definida")

## 📊 Procesar Crohn

In [ ]:
print("📊 Cargando datos de Crohn...\n")
df_crohn = pd.read_csv('../data/processed/crohn_filtered.csv')
df_crohn['checkin_date'] = pd.to_datetime(df_crohn['checkin_date'])

print(f"✓ Cargado: {len(df_crohn):,} registros")
print(f"✓ Usuarios: {df_crohn['user_id'].nunique():,}")

In [ ]:
# Procesar Crohn
crohn_dataset, crohn_metadata = process_ibd_data(
    df_crohn,
    ibd_type='crohn',
    symptom_weights=SYMPTOM_WEIGHTS_CROHN
)

In [ ]:
# Guardar Crohn
print("\n💾 Guardando dataset de Crohn...")
crohn_dataset.to_csv('../data/processed/crohn/ml_dataset.csv', index=False)

with open('../data/processed/crohn/ml_dataset_metadata.json', 'w') as f:
    json.dump(crohn_metadata, f, indent=2)

print(f"✅ Crohn dataset guardado:")
print(f"  - ../data/processed/crohn/ml_dataset.csv ({len(crohn_dataset):,} registros)")
print(f"  - ../data/processed/crohn/ml_dataset_metadata.json")

## 📊 Procesar Colitis Ulcerosa

In [ ]:
print("\n📊 Cargando datos de CU...\n")
df_cu = pd.read_csv('../data/processed/cu_filtered.csv')
df_cu['checkin_date'] = pd.to_datetime(df_cu['checkin_date'])

print(f"✓ Cargado: {len(df_cu):,} registros")
print(f"✓ Usuarios: {df_cu['user_id'].nunique():,}")

In [ ]:
# Procesar CU
cu_dataset, cu_metadata = process_ibd_data(
    df_cu,
    ibd_type='cu',
    symptom_weights=SYMPTOM_WEIGHTS_UC
)

In [ ]:
# Guardar CU
print("\n💾 Guardando dataset de CU...")
cu_dataset.to_csv('../data/processed/cu/ml_dataset.csv', index=False)

with open('../data/processed/cu/ml_dataset_metadata.json', 'w') as f:
    json.dump(cu_metadata, f, indent=2)

print(f"✅ CU dataset guardado:")
print(f"  - ../data/processed/cu/ml_dataset.csv ({len(cu_dataset):,} registros)")
print(f"  - ../data/processed/cu/ml_dataset_metadata.json")

## 📊 Comparación Final

In [ ]:
print("\n" + "="*80)
print("RESUMEN COMPARATIVO")
print("="*80)

print(f"\n📊 CROHN:")
print(f"  Registros: {len(crohn_dataset):,}")
print(f"  Usuarios: {crohn_metadata['n_users']:,}")
print(f"  Distribución:")
for level, count in crohn_metadata['class_distribution'].items():
    pct = count / len(crohn_dataset) * 100
    print(f"    {level}: {count:,} ({pct:.1f}%)")

print(f"\n📊 CU:")
print(f"  Registros: {len(cu_dataset):,}")
print(f"  Usuarios: {cu_metadata['n_users']:,}")
print(f"  Distribución:")
for level, count in cu_metadata['class_distribution'].items():
    pct = count / len(cu_dataset) * 100
    print(f"    {level}: {count:,} ({pct:.1f}%)")

print("\n" + "="*80)
print("✅ FEATURE ENGINEERING V2 COMPLETADO")
print("="*80)
print("\nPróximos pasos:")
print("  1. Notebook 04: Entrenar modelos cluster-stratified para Crohn")
print("  2. Notebook 05: Entrenar modelos cluster-stratified para CU")

## 📈 Visualización Comparativa

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Crohn distribution
crohn_counts = crohn_dataset['risk_level'].value_counts()
colors_dict = {'low': 'green', 'medium': 'orange', 'high': 'red'}
axes[0].bar(crohn_counts.index, crohn_counts.values,
           color=[colors_dict.get(x, 'gray') for x in crohn_counts.index],
           edgecolor='black')
axes[0].set_title('Distribución Risk Levels - CROHN', fontweight='bold', fontsize=12)
axes[0].set_xlabel('Risk Level')
axes[0].set_ylabel('Frecuencia')
axes[0].grid(True, alpha=0.3, axis='y')

# CU distribution
cu_counts = cu_dataset['risk_level'].value_counts()
axes[1].bar(cu_counts.index, cu_counts.values,
           color=[colors_dict.get(x, 'gray') for x in cu_counts.index],
           edgecolor='black')
axes[1].set_title('Distribución Risk Levels - CU', fontweight='bold', fontsize=12)
axes[1].set_xlabel('Risk Level')
axes[1].set_ylabel('Frecuencia')
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('../docs/figures/risk_distribution_crohn_vs_cu.png', dpi=300, bbox_inches='tight')
plt.show()

print("💾 Figura guardada: ../docs/figures/risk_distribution_crohn_vs_cu.png")